# 飞桨常规赛：PALM眼底彩照视盘探测与分割-7月第2名方案

## 赛题介绍
本赛题原型为ISBI2019PALM眼科大赛。 近视已成为全球公共卫生负担。在近视患者中，约35%为高度近视。近视导致眼轴长度的延长，可能引起视网膜和脉络膜的病理改变。随着近视屈光度的增加，高度近视将发展为病理性近视，其特点是病理改变的形成:(1)后极，包括镶嵌型眼底、后葡萄肿、视网膜脉络膜变性等;(2)视盘，包括乳头旁萎缩、倾斜等;(3)近视性黄斑，包括漆裂、福氏斑、CNV等。病理性近视对患者造成不可逆的视力损害。因此，早期诊断和定期随访非常重要。

视网膜由黄斑向鼻侧约3mm处有一直径约1.5mm、境界清楚的淡红色圆盘状结构，称为视神经盘，简称视盘。视盘是眼底图像的一个重要特征，对其进行准确、快速地定位与分割对利用眼底图像进行疾病辅助诊断具有重要意义。

- $\color{red}{注意：本次比赛需要选手使用环境：飞桨PaddlePaddle 2.1.0版本}$

- $\color{red}{注意：AI Studio上运行需要使用32G内存的高级版，本地运行同样需要配置较大的内存空间。}$

### [点击跳转至赛题页面](https://aistudio.baidu.com/aistudio/competition/detail/87)

## 解题思路
本文通过PaddleSeg的配置方式使用UNet模型完成该任务。
PaddleSeg是基于飞桨PaddlePaddle开发的端到端图像分割开发套件，涵盖了高精度和轻量级等不同方向的大量高质量分割模型。通过模块化的设计，提供了配置化驱动和API调用两种应用方式，帮助开发者更便捷地完成从训练到部署的全流程图像分割应用。
![](https://ai-studio-static-online.cdn.bcebos.com/e9d5ad1086a045828a04be3e93148bb918e38ff2405743dd89d4b7e5e6ff05d0)

图1 UNet结构图（图片来源：https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.2/docs/models/unet.md）


### 配置项 [官方文档](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.2/docs/design/use/use_cn.md)
- $\color{red}{注意：本文使用自定义数据集，配置文件中train_dataset和val_dataset的属性 type: Dataset。}$
----
### train_dataset
* 训练数据集
----
### val_dataset
* 验证数据集

----
### batch_size
* 单张卡上，每步迭代训练时的数据量。一般来说，你所使用机器的显存越大，可以相应的调高batch_size的值。

----
### iters
* 使用一个 batch 数据对语义分割模型进行一次参数更新的过程称之为一次训练，即一次迭代。iters 即为训练过程中的迭代次数。

----
### optimizer
* 训练优化器

----
### lr_scheduler
* 学习率

----
### loss
* 损失函数

----
### model
* 待训练模型
---
### export
* 模型导出配置

# 环境初始化

#### 主要工作：解压文件，删除无用文件夹，创建训练所需文件夹

In [ ]:
# 解压文件
!unzip -oq /home/aistudio/data/data101358/常规赛：PALM眼底彩照视盘探测与分割.zip -d /home/aistudio/data/

# 删除无用文件夹
!rm -rf /home/aistudio/data/__MACOSX/

# 解压工具包
!rm -rf /home/aistudio/PaddleSeg
!unzip -oq /home/aistudio/data/data102868/PaddleSeg.zip -d /home/aistudio


# 创建数据集文件夹
!rm -rf /home/aistudio/dataset
!mkdir /home/aistudio/dataset

# 把数据移动到对应的目录
!mv /home/aistudio/data/常规赛：PALM眼底彩照视盘探测与分割/Train/Disc_Masks /home/aistudio/dataset/labels

# 创建原始文件文件夹
!mv /home/aistudio/data/常规赛：PALM眼底彩照视盘探测与分割/Train/fundus_image /home/aistudio/dataset/images

# 创建测试文件夹
!mv /home/aistudio/data/常规赛：PALM眼底彩照视盘探测与分割/PALM-Testing400-Images /home/aistudio/dataset/test

# 删除无用的文件夹
!rm -rf /home/aistudio/data/常规赛：PALM眼底彩照视盘探测与分割
!rm -rf /home/aistudio/dataset/labels/.DS_Store
!rm -rf /home/aistudio/dataset/images/.DS_Store
!rm -rf /home/aistudio/dataset/test/.DS_Store

# 文件初始化

#### 主要工作：划分训练集和验证集，生成训练集、验证集、测试集文件列表
#### 其中训练集和验证集列表格式：
path_of_train_file path_of_label_file_name

文件名之间用英文空格分开

In [ ]:
%cd /home/aistudio
# 划分验证集和训练集 并生成文件列表
!python utils/create_train_list.py

# 处理标注图片
#### 主要工作：将标注图片转成黑白图片

In [ ]:
# 将标注文件改为黑白标注
%cd /home/aistudio
!python utils/create_gray_labels.py

# 安装所需包

In [1]:
# 安装所需包 去掉输出信息
!pip install -qr /home/aistudio/PaddleSeg/requirements.txt

# 执行训练

#### 本文使用配置模式完成训练和预测

In [13]:
# 执行训练
%cd /home/aistudio/PaddleSeg

!export CUDA_VISIBLE_DEVICES=0 # 设置1张可用的卡

!python train.py \
       --config /home/aistudio/utils/unet.yml \
       --do_eval \
       --use_vdl \
       --save_interval 200 \
       --save_dir output

# 对测试图片进行预测
### PaddleSeg在预测时指定测试集文件路径即可完成该目录下所有测试文件的预测
### 参数 save_dir 表示预测文件输出目录

In [15]:
# 执行预测
%cd /home/aistudio/PaddleSeg

!python predict.py \
       --config /home/aistudio/utils/unet.yml \
       --model_path output/best_model/model.pdparams\
       --image_path /home/aistudio/dataset/test \
       --save_dir output/result

# 对文件进行转换
### 把预测图片转换为黑白图片

In [2]:
# 文件转换为黑白图片
%cd /home/aistudio/
!python utils/resave_predict_images.py

# 预测文件打包
### 将预测结果进行打包提交结果

In [3]:
# 文件打包
%cd /home/aistudio/dataset/
!cp -r /home/aistudio/PaddleSeg/output/result/pseudo_color_prediction/ ./predicts
!zip -r pseudo_color_prediction.zip ./predicts

# 下一步计划及作者介绍

1、修改配置参数对比预测效果

2、测试不用的模型对比预测结果

3、试用数据集创建脚本进一步提高自动化程度

4、测试接口开发方式

# 作者介绍

#### 胡志新（xiaohuAiTec） 中国青年报社技术开发

#### 研究方向：主要研究自然语言处理，偶尔关注计算机视觉


#### 我在AI Studio上获得黄金等级，点亮4个徽章，来互关呀~

#### [https://aistudio.baidu.com/aistudio/personalcenter/thirdview/638](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/638)


![](https://ai-studio-static-online.cdn.bcebos.com/f209e0c2681b4e06b3c6d66844f10a85c168062d3dc04859a6fe9a2acc57a150)
